# Advent of Code 2021: Day 5
link to puzzle [here](https://adventofcode.com/2021/day/5)

Imports

In [1]:
from dataclasses import dataclass, field
from typing import List

import numpy as np
import numpy.typing as npt

!ln -sf ../utils.py .
import utils

Load puzzle input data

In [2]:
DATA_DIR = '../data'
DAY = 5
data = utils.get_puzzle_input(day=DAY, input_dir=DATA_DIR)

Functions

In [180]:
def parse_endpoints(s):
    """map strings of the form `'x1,y1 -> x2,y2'` 
    to 2x2 integer arrays of the form `[[x1, y1], [x2, y2]]`
    """
    return np.array([pt.split(',') for pt in s.split('->')], dtype=int)

def get_all_endpoints(data):
    """iterate through the puzzle input to parse all endpoints
    and return a list of 2x2 integer arrays
    """
    return [parse_endpoints(s) for s in data]

def is_horizontal(endpoints):
    """return True if the endpoints define a horizontal line segment, 
    i.e. the y values of each enpoint are equal, otherwise return False
    """
    return endpoints[0, 1] == endpoints[1, 1]

def is_vertical(endpoints):
    """return True if the endpoints define a vertical line segment, 
    i.e. the x values of each enpoint are equal, otherwise return False
    """
    return endpoints[0, 0] == endpoints[1, 0]

def integer_line_fill(endpoints: np.ndarray, diagonal: bool=False):
    """given a 2x2 numpy array `endpoints` whose rows are endpoints of a line segment L, return a
    n x 2 array of points [x, y] such that [x, y] lie on L and x, y are integers
    """    
    if is_vertical(endpoints):
        dy = endpoints[0, 1] - endpoints[1, 1]
        x = np.repeat(endpoints[0, 0], abs(dy) + 1)
        y = range(endpoints[:, 1].min(), endpoints[:, 1].max() + 1)
        line = np.stack([x, y], axis=1)
    elif is_horizontal(endpoints):
        dx = endpoints[0, 0] - endpoints[1, 0]
        x = range(endpoints[:, 0].min(), endpoints[:, 0].max() + 1)
        y = np.repeat(endpoints[0, 1], abs(dx) + 1)
        line = np.stack([x, y], axis=1)
    else:
        xstep = -1 if endpoints[0, 0] > endpoints[1, 0] else 1
        ystep = -1 if endpoints[0, 1] > endpoints[1, 1] else 1
        x = range(endpoints[0, 0], endpoints[1, 0], xstep)
        y = range(endpoints[0, 1], endpoints[1, 1], ystep)
        line = np.vstack([np.stack([x,y], axis=1), endpoints[1, :]])
    return line

def count_vent_overlaps(endpoints_list, skip_diagonal_lines=True):
    # initialize grid of zeros, grid will iteratively update values a_ij = a_ij + 1
    # when a line defined by a pair of endpoints contains the point (i, j)
    nrows = max([ep[:, 0].max() for ep in endpoints_list]) + 1
    ncols = max([ep[:, 1].max() for ep in endpoints_list]) + 1
    grid = np.zeros((nrows, ncols))

    # for each set of endpoint pairs (p1, p1)
    #     1. Construct the line segment L defined by (p1, p2)
    #     2. For each integer point (x, y) lying on L,
    #        add 1 to the element of grid at row x, column y
    # then the resulting grid values will counts the number of
    # line segments which intersect that point
    for endpoints in endpoints_list:
        line = integer_line_fill(endpoints)
        if is_vertical(endpoints):
            x, y = line[0, 0], line[:, 1] 
        elif is_horizontal(endpoints):
            x, y = line[:, 0], line[0, 1] 
        else:
            if skip_diagonal_lines:  
                continue
            else:
                x, y = line[:, 0], line[:, 1]
        # update grid counts for new line if horizontal or vertical
        grid[x, y] += 1
    return grid

In [181]:
endpoints_list = get_all_endpoints(data)
grid = count_vent_overlaps(endpoints_list)

# count points in grid which intersect > 1 line segment
print((grid > 1).sum())

7142


#### Part 2

In [182]:
endpoints_list = get_all_endpoints(data)
grid = count_vent_overlaps(endpoints_list, skip_diagonal_lines=False)

# count points in grid which intersect > 1 line segment
print((grid > 1).sum())

20012
